In [1]:
import pandas as pd

In [2]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

# Q1. Downloading the data

In [3]:
january_trips = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-01.parquet")
february_trips = pd.read_parquet("https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-02.parquet")

- Read the data for January. How many columns are there?

In [4]:
len(january_trips.columns)

19

# Q2. Computing duration

In [5]:
january_trips.head()

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,fare_amount,extra,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,airport_fee
0,2,2023-01-01 00:32:10,2023-01-01 00:40:36,1.0,0.97,1.0,N,161,141,2,9.3,1.00,0.5,0.00,0.0,1.0,14.30,2.5,0.00
1,2,2023-01-01 00:55:08,2023-01-01 01:01:27,1.0,1.10,1.0,N,43,237,1,7.9,1.00,0.5,4.00,0.0,1.0,16.90,2.5,0.00
2,2,2023-01-01 00:25:04,2023-01-01 00:37:49,1.0,2.51,1.0,N,48,238,1,14.9,1.00,0.5,15.00,0.0,1.0,34.90,2.5,0.00
3,1,2023-01-01 00:03:48,2023-01-01 00:13:25,0.0,1.90,1.0,N,138,7,1,12.1,7.25,0.5,0.00,0.0,1.0,20.85,0.0,1.25
4,2,2023-01-01 00:10:29,2023-01-01 00:21:19,1.0,1.43,1.0,N,107,79,1,11.4,1.00,0.5,3.28,0.0,1.0,19.68,2.5,0.00


In [6]:
def compute_duration(df):
    # df = df.copy()  # Create a copy of the DataFrame to avoid modifying the original one

    # Calculate the duration
    df["duration"] = df["tpep_dropoff_datetime"] - df["tpep_pickup_datetime"]
    df["duration"] = df["duration"].apply(lambda td: td.total_seconds() / 60)
    
    return df


In [7]:
january_trips = compute_duration(january_trips)
february_trips = compute_duration(february_trips)

- What's the standard deviation of the trips duration in January?

In [8]:
january_trips["duration"].std()

42.594351241920904

# Q3. Dropping outliers

In [9]:
total_records_before = len(january_trips)

In [10]:
january_trips = january_trips[(january_trips["duration"] >= 1) & (january_trips["duration"] <= 60)]

In [11]:
total_records_after = len(january_trips)

- What fraction of the records left after you dropped the outliers?

In [12]:
fraction_remaining = total_records_after / total_records_before
fraction_remaining

0.9812202822125979

# Q4. One-hot encoding

In [13]:
dv = DictVectorizer()

january_trips['PULocationID'] = january_trips['PULocationID'].astype(str)
january_trips['DOLocationID'] = january_trips['DOLocationID'].astype(str)

train_dicts = january_trips[['PULocationID', 'DOLocationID', 'trip_distance']].to_dict(orient='records')

X_train = dv.fit_transform(train_dicts)

- What's the dimensionality of this matrix (number of columns)?

In [14]:
num_columns = X_train.shape[1]
num_columns

516

# Q5. Training a model

In [15]:
y_train = january_trips["duration"].values

In [16]:
lr = LinearRegression()
lr.fit(X_train, y_train)

LinearRegression()

In [17]:
y_pred = lr.predict(X_train)
mean_squared_error(y_train, y_pred, squared=False)

7.658406632748288

# Q6. Evaluating the model

In [18]:
february_trips['PULocationID'] = february_trips['PULocationID'].astype(str)
february_trips['DOLocationID'] = february_trips['DOLocationID'].astype(str)

val_dicts = february_trips[['PULocationID', 'DOLocationID', 'trip_distance']].to_dict(orient='records')

X_val = dv.transform(val_dicts)

In [19]:
y_val = february_trips["duration"].values

lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

42.28816859376711